In [10]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, curve_fit
import matplotlib.pyplot as plt
import pyodbc
from datetime import date
import streamlit as st
import altair as alt
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
def arps_decline(t, Qi, Dei, b, Def, prior_cum=0, prior_t=0):
    # UID is a unique identifier for the well such as API, must be a number
    # phase is 1 = oil, 2 = gas, or 3 = water
    # Qi is the initial production rate typically in bbl/day or Mcf/day
    # Dei is the initial effective annual decline rate
    # Def is the final effective annual decline rate at which point the decline becomes exponential
    # b is the b-factor used in hyperbolic or harmonic decline equations
    # t is the time as a month integer
    # prior_cum is the cumulative amount produced before the start of the decline calcuations
    # prior_t is an integer representing the final month from a previous decline segment
    
    # Calculations to determine decline type
    if Dei == Def:
        Type = 'exp'
    elif Dei > Def and b == 1:
        Type = 'har'
        Dn = Dei / (1 - Dei)
        Qlim = Qi * ((-np.log(1 - Def)) / Dn)
        tlim = (((Qi / Qlim) - 1) / Dn) * 12 # output in months
    else:
        Type = 'hyp'
        Dn = (1 / b) * (((1 - Dei) ** -b) - 1)
        Qlim = Qi * ((-np.log(1 - Def)) / Dn) ** (1 / b)
        tlim = ((((Qi / Qlim) ** b) - 1) / ( b * Dn)) * 12 # output in months
    
    # Generate volumes
    if Type == 'hyp':
        Dn_t = Dn / (1 + b * Dn * (t / 12))
        De_t = 1 - (1 / ((Dn_t * b) + 1)) ** (1 / b)
        if De_t > Def:
            q = Qi * (1 + b * Dn * (t / 12)) ** (-1/b)
            Np = ((Qi ** b) / (Dn * (1 - b))) * ((Qi ** (1 - b)) - (q ** (1 - b))) * 365
        else:
            q = Qlim * np.exp(-(-np.log(1 - Def)) * ((t - tlim) / 12))
            Np = ((Qlim - q) / (-np.log(1 - Def)) * 365) + (((Qi ** b) / 
                    (Dn * (1 - b))) * ((Qi ** (1 - b)) - (Qlim ** (1 - b))) * 365)
            De_t = Def
    elif Type == 'har':
        Dn_t = Dn / (1 + Dn * (t / 12))
        De_t = 1 - (1 / (Dn_t + 1))
        if De_t > Def:
            q = Qi / (1 + b * Dn * (t / 12))
            Np = (Qi / Dn) * np.log(Qi / q) * 365
        else:
            q = Qlim * np.exp(-(-np.log(1 - Def)) * ((t - tlim) / 12))
            Np = ((Qlim - q) / (-np.log(1 - Def)) * 365) + ((Qi / Dn) * np.log(Qi / Qlim) * 365)
            De_t = Def
    else:
        q = Qi * np.exp(-(-np.log(1 - Dei)) * (t / 12))
        Np = (Qi - q) / (-np.log(1 - Dei)) * 365
        De_t = Dei
    
    return t + prior_t, q, Np + prior_cum

In [3]:
varps_decline = np.vectorize(arps_decline)

In [4]:
# Prepare SQL statement
SELECT = 'SELECT P.Property_ID, P.Date, P.Oil, P.Gas, DATEDIFF(DD, W.First_Prod_Date, P.Date) AS date_diff '
FROM = 'FROM dbo.Production P '
JOIN = 'INNER JOIN (SELECT Property_ID, First_Prod_Date FROM dbo.Wells) W ON P.Property_ID = W.Property_ID'

In [5]:
# Bring in production data from SQL
def prod_data(db_name = 'SCST_Wells', server_name = 'DESKTOP-A3I0ATB'):
    conn_stmt = f'DRIVER={{SQL Server}};SERVER={server_name};DATABASE={db_name};Trusted_Connection=yes'
    conn = pyodbc.connect(conn_stmt, autocommit=True)
    cursor = conn.cursor()
    sql = SELECT + FROM + JOIN
    cursor.execute(sql)
    data = cursor.fetchall()
    prod_df = pd.DataFrame.from_records(data, columns=[col[0] for col in cursor.description])
    return prod_df

In [6]:
# Create dataframe with production data and another to create well lists for fitting
prod_df = prod_data()
prod_pivot_df = prod_df.groupby(['Property_ID']).agg({'Oil': 'count', 'Gas': 'count'}).reset_index()

In [7]:
# Create lists of wells to filter down the well set.
# Purpose is to understand how decline fits evolve with more data
oil_list = list(prod_pivot_df[(prod_pivot_df['Oil'] >= 42) & (prod_pivot_df['Oil'] <= 60)]['Property_ID'].values)
gas_list = list(prod_pivot_df[(prod_pivot_df['Gas'] >= 42) & (prod_pivot_df['Gas'] <= 60)]['Property_ID'].values)

In [8]:
# Add some columns
prod_df = prod_df.astype({'Oil': float, 'Gas': float})
prod_df['Date'] = pd.to_datetime(prod_df['Date'], format='%Y-%m-%d')
prod_df['day'] = prod_df.Date.dt.day
prod_df['daily_oil'] = prod_df['Oil'] / prod_df['day']
prod_df['daily_gas'] = prod_df['Gas'] / prod_df['day']
prod_df = prod_df.fillna(0)
prod_df.drop(['day'], axis=1, inplace=True)

In [11]:
# Regression equation for Dei to prepare bounds
prod_df['min_oil_dei'] = 0.01 * np.log(prod_df['Oil']) - 0.169 * np.log(prod_df['date_diff'] + 1) + 1.536
prod_df['exp_oil_dei'] = 0.0139 * np.log(prod_df['Oil']) - 0.1603 * np.log(prod_df['date_diff'] + 1) + 1.6118
prod_df['max_oil_dei'] = 0.018 * np.log(prod_df['Oil']) - 0.152 * np.log(prod_df['date_diff'] + 1) + 1.687
prod_df['min_gas_dei'] = 0.01 * np.log(prod_df['Gas']) - 0.169 * np.log(prod_df['date_diff'] + 1) + 1.536 - 0.159
prod_df['exp_gas_dei'] = 0.0139 * np.log(prod_df['Gas']) - 0.1603 * np.log(prod_df['date_diff'] + 1) + 1.6118 - 0.1464
prod_df['max_gas_dei'] = 0.018 * np.log(prod_df['Gas']) - 0.152 * np.log(prod_df['date_diff'] + 1) + 1.687 - 0.133

In [13]:
# Define parameters
b_init = 1.1
Def = 0.08
min_q = 10.0
max_fit_months = [36, 24, 12, 6] # represents months of tail of production data
param_df_cols = ['Property_ID', 'months_of_prod', 'fit_months', 'fit_type', 'Q_guess', 'Q3', 'Dei', 'b-factor']

# Define function for fitting
def arps_fit(t, Qi, Dei, b):
    return varps_decline(t, Qi, Dei, b, Def)[1]

# loop through lists
param_df = pd.DataFrame()
for w in oil_list:
    for m in max_fit_months:
        df = prod_df[(prod_df['Property_ID'] == w) & (prod_df['Oil'] > 0)]
        arr_length = len(df)
        fit_length = min(arr_length, m)
        df = df.tail(fit_length)
        t_act = df['Date'].rank(method='min', ascending=True).values
        q_act = df['daily_oil'].values
        Qi_guess = np.max(q_act)
        Dei_init = np.max([np.min([df.iloc[0, 7], 0.99]), Def])
        Dei_min = np.max([np.min([df.iloc[0, 8], 0.99]), Def])
        Dei_max = np.max([np.min([df.iloc[0, 9], 0.99]), Def])

        # fit Arps equation
        if Qi_guess < min_q:
            param_list = [w, arr_length, m, 'auto_fit_4', Qi_guess, Qi_guess, Dei_init, b_init]
        else:
            try:
                bounds = ((Qi_guess*0.9, Dei_min, 0.3), (Qi_guess, Dei_max, 1.2))
                popt, pcov = curve_fit(arps_fit, t_act, q_act, p0=[Qi_guess, Dei_init, b_init], bounds=bounds, maxfev=3000)
                qi_fit, Dei_fit, b_fit = popt
                param_list = [w, arr_length, m, 'auto_fit_1', Qi_guess, qi_fit, Dei_fit, b_fit]
            except:
                try:
                    bounds = ((Qi_guess*0.9, Qi_guess))
                    popt, pcov = curve_fit(arps_fit, t_act, q_act, p0=[Qi_guess], bounds=bounds, maxfev=3000)
                    qi_fit = popt[0]
                    param_list = [w, arr_length, m, 'auto_fit_2', Qi_guess, qi_fit, Dei_init, b_init]
                except:
                    param_list = [w, arr_length, m, 'auto_fit_3', Qi_guess, Qi_guess, Dei_init, b_init]
        df_i = pd.DataFrame([param_list], columns=param_df_cols)
        param_df = pd.concat([param_df, df_i]).reset_index(drop=True)

In [ ]:
param_df.to_csv('arps_param_autofits.csv')